In [ ]:
import cv2
import mediapipe as mp
import vlc
import time
import math

# ----------- Setup VLC & Playlist -----------
vlc_instance = vlc.Instance()
playlist = ['song1.mp3', 'song2.mp3', 'song3.mp3']  # ← مسیرهای شما
media_list = vlc_instance.media_list_new(playlist)

list_player = vlc_instance.media_list_player_new()
list_player.set_media_list(media_list)

# وضعیت پخش
current_index = 0
last_volume = 50
player_rate = 1.0

def play_index(idx):
    global current_index, player
    current_index = idx % len(playlist)
    list_player.play_item_at_index(current_index)
    player = list_player.get_media_player()
    player.audio_set_volume(last_volume)
    player.set_rate(player_rate)

play_index(current_index)

hands = mp.solutions.hands.Hands(
    static_image_mode=False, max_num_hands=2, model_complexity=1,
    min_detection_confidence=0.7, min_tracking_confidence=0.7
)
face = mp.solutions.face_mesh.FaceMesh(
    static_image_mode=False, refine_landmarks=False, max_num_faces=1,
    min_detection_confidence=0.7, min_tracking_confidence=0.7
)

def dist(a, b):
    return math.hypot(a[0] - b[0], a[1] - b[1])

last_head = 0
head_cooldown = 1.0  
alpha = 0.2


cap = cv2.VideoCapture(1)
# cap = cv2.VideoCapture("http://192.168.254.7:4747")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    now = time.time()

    hr = hands.process(rgb)
    fr = face.process(rgb)

    # ----- کنترل ولوم و سرعت با دست -----
    if hr.multi_hand_landmarks:
        for info, lmks in zip(hr.multi_handedness, hr.multi_hand_landmarks):
            label = info.classification[0].label
            pts = [(int(p.x * w), int(p.y * h)) for p in lmks.landmark]
            x8, y8 = pts[8]

            if w*0.2 < x8 < w*0.8:
                d = dist(pts[4], pts[8])
                if label == 'Right':
                    vol = max(0, min(100, int(d/(w/3)*100)))
                    player.audio_set_volume(vol)
                    last_volume = vol
                    cv2.putText(frame, f'Vol: {vol}', (x8, y8-20),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
                else:
                    target = max(0.1, min(3.0, d/(w/3)*3.0))
                    sm = player_rate + (target-player_rate)*alpha
                    player.set_rate(sm)
                    player_rate = sm
                    cv2.putText(frame, f'Rate: {player_rate:.2f}x',
                                (x8, y8-20), cv2.FONT_HERSHEY_SIMPLEX, 1,
                                (0,255,0), 2)

    # ----- چرخش سر برای ترک بعدی/قبلی و بالا/پایین -----
    if fr.multi_face_landmarks and now - last_head > head_cooldown:
        lm = fr.multi_face_landmarks[0].landmark
        nose = lm[1]  # نقطه‌ی 1 معمولاً نوک بینی
        nx, ny = int(nose.x * w), int(nose.y * h)
        mid_x, mid_y = w//2, h//2
        thr_x, thr_y = w*0.15, h*0.15

        # بعدی/قبلی محور افقی
        if nx > mid_x + thr_x:
            play_index(current_index + 1)
            last_head = now
        elif nx < mid_x - thr_x:
            play_index(current_index - 1)
            last_head = now

        # بالا/پایین محور عمودی
        elif ny < mid_y - thr_y:
            # سر به بالا → توقف پخش
            if player.is_playing():
                player.pause()
                last_head = now
        elif ny > mid_y + thr_y:
            # سر به پایین → ادامه پخش (اگر در حالت توقف باشد)
            if not player.is_playing():
                player.play()
                last_head = now

    # نمایش فریم
    cv2.imshow('Music Player', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 

In [2]:
import cv2

available = []
for idx in range(6):
    cap = cv2.VideoCapture(idx)
    if cap.isOpened():
        ret, _ = cap.read()
        if ret:
            available.append(idx)
        cap.release()

print("Available camera indices:", available)


Available camera indices: [0, 1, 2, 3]
